In [1]:
import sys
sys.path.append('../src/')
import time
import os
import copy

import numpy as np
import tensorly as tn
import matlab.engine

from loewnerize_acts import loewnerize_acts
import ll1_tools

## Load raw data for LenetLinear classifier model

In [4]:
targets_path = '../data/LenetLinear/activations/test/targets.npy'
targets = np.load(targets_path)

inputs_path = '../data/LenetLinear/activations/test/99/layers.layer_0.linear.npy'
inputs_raw = np.load(inputs_path)
print('Inputs shape')
print(inputs_raw.shape)

outputs_path = '../data/LenetLinear/activations/test/99/layers.layer_0.rat.npy'
outputs_raw = np.load(outputs_path)
print('Outputs shape')
print(outputs_raw.shape)
print(outputs_raw[0,0,0])

# tensorlab_path = '~/ware/matlab_tools/tensorlab'
tensorlab_path = '~/tools/matlab_tools/Tensorlab'

Inputs shape
(39, 256, 10)
Outputs shape
(39, 256, 10)
-5.9573436


In [5]:
lwn_tensors = loewnerize_acts(inputs_raw, outputs_raw, targets, tensorlab_path)
# datapath = '../data/LenetLinear/activations/test/99/loewnerized_acts'
# np.savez(datapath, *lwn_tensors)

In [6]:
for lwn_trgt in lwn_tensors:
    print(lwn_trgt.shape)
    print(type(lwn_trgt))

(5, 5, 979)
<class 'numpy.ndarray'>
(5, 5, 1133)
<class 'numpy.ndarray'>
(5, 5, 1030)
<class 'numpy.ndarray'>
(5, 5, 1008)
<class 'numpy.ndarray'>
(5, 5, 980)
<class 'numpy.ndarray'>
(5, 5, 890)
<class 'numpy.ndarray'>
(5, 5, 956)
<class 'numpy.ndarray'>
(5, 5, 1027)
<class 'numpy.ndarray'>
(5, 5, 973)
<class 'numpy.ndarray'>
(5, 5, 1008)
<class 'numpy.ndarray'>


## perform (L,L,1) decomposition - per target (0-9)

In [7]:
eng = matlab.engine.start_matlab()
s = eng.genpath(tensorlab_path)
eng.addpath(s, nargout=0)

num_terms = 20
rat_deg = 3

L = rat_deg * np.ones(num_terms)
Us = list()
outs = list()

# tic = time.perf_counter()
# for tns in lwn_tensors:
#     tmp_U, tmp_out = eng.ll1(tns, L, nargout=2)
#     Us.append(tmp_U)
#     outs.append(tmp_out)
# toc = time.perf_counter()
# print(toc-tic)

## perform (L,L,1) decomposition - entire collection

In [8]:
super_lwn = lwn_tensors[0]
print(type(super_lwn))
for i in range(9):
    super_lwn = np.dstack((super_lwn, lwn_tensors[i+1]))

# shuffle
np.random.shuffle(super_lwn)
print(super_lwn.shape)

super_lwn_1 = np.array(super_lwn[:,:,:(9984//2)])
super_lwn_2 = np.array(super_lwn[:,:,(9984//2):])

print(super_lwn_1.shape)
print(super_lwn_2.shape)


<class 'numpy.ndarray'>
(5, 5, 9984)
(5, 5, 4992)
(5, 5, 4992)


In [9]:
num_terms = 25
rat_deg = 3
L = rat_deg * np.ones(num_terms)

tic = time.perf_counter()
all_U_1, all_out_1 = eng.ll1(super_lwn_1, L, nargout=2)
all_U_2, all_out_2 = eng.ll1(super_lwn_2, L, nargout=2)

toc = time.perf_counter()
print(toc-tic)

> In ll1_core/state (line 238)
In ll1_core (line 191)
In ll1_nls (line 40)
In ll1 (line 551)

> In ll1_core/state (line 238)
In ll1_core/grad (line 256)
In nls_gndl (line 369)
In ll1_core (line 193)
In ll1_nls (line 40)
In ll1 (line 551)

> In ll1_core/state (line 238)
In ll1_core/grad (line 256)
In nls_gndl (line 369)
In ll1_core (line 193)
In ll1_nls (line 40)
In ll1 (line 551)

> In ll1_core/state (line 238)
In ll1_core/grad (line 256)
In nls_gndl (line 369)
In ll1_core (line 193)
In ll1_nls (line 40)
In ll1 (line 551)

> In ll1_core/state (line 238)
In ll1_core/grad (line 256)
In nls_gndl (line 369)
In ll1_core (line 193)
In ll1_nls (line 40)
In ll1 (line 551)

> In ll1_core/state (line 238)
In ll1_core/grad (line 256)
In nls_gndl (line 369)
In ll1_core (line 193)
In ll1_nls (line 40)
In ll1 (line 551)

> In ll1_core/state (line 238)
In ll1_core/grad (line 256)
In nls_gndl (line 369)
In ll1_core (line 193)
In ll1_nls (line 40)
In ll1 (line 551)

> In ll1_core/state (line 238)
In ll

In [17]:
res_1 = 1 - eng.frobll1res(super_lwn_1,all_U_1) / eng.frob(super_lwn_1)
print(f"Residual_1: {res_1}")
res_2 = 1 - eng.frobll1res(super_lwn_2,all_U_2) / eng.frob(super_lwn_2)
print(f"Residual_2: {res_2}")

Residual_1: 0.9999989875639654
Residual_2: 0.9999974078605245


In [18]:
print("time to party")


time to party
